In [50]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
 
img_width, img_height = 128, 128

input_shape = (img_width, img_height, 3) ## we checked that the RGB channel comes last in the data format of these images

epochs = 10
batch_size = 16

### Creating the model (convolutional neural net)

In [49]:
model = Sequential() 

# first convolutional layer 
model.add(Conv2D(32, (4, 4), activation = 'relu', input_shape = input_shape)) ## convolution of kernel with image
model.add(MaxPooling2D(pool_size=(2, 2))) ## reduces dimensionality in order to reduce computational cost + minimize overfitting

# second convolutional layer
model.add(Conv2D(32, (4, 4), activation = 'relu'))  ## convolution of kernel with image
model.add(MaxPooling2D(pool_size=(2, 2)))  ## reduces dimensionality in order to reduce computational cost + minimize overfitting
 
# third convolutional layer
model.add(Conv2D(64, (4, 4), activation = 'relu'))  ## convolution of kernel with image
model.add(MaxPooling2D(pool_size=(2, 2)))  ## reduces dimensionality in order to reduce computational cost + minimize overfitting

model.add(Flatten()) ## transitions from convolutional to fully connected layers by converting the input from multi-dimensional --> 1D

# first densely-connected layer 
model.add(Dense(64, activation = 'relu')) ## creates a fully connected layer with 64 output units (neurons)
model.add(Dropout(0.2)) ## this function randomly sets 20% of the input units to 0 at each training step; used to prevent overfitting

# second densely-connected layer 
model.add(Dense(16, activation = 'relu')) ## creates a fully connected layer with 64 output units (neurons)
model.add(Dropout(0.2)) ## this function randomly sets 20% of the input units to 0 at each training step; used to prevent overfitting

# third densely-connected layer 
model.add(Dense(4, activation = 'sigmoid')) ## creates a fully connected layer with 4 output units (neurons) corresponding to the 4 output classes

model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy']) ## try accuracy for now, can change later

### Building the model (training on train data and building it to maximize accuracy on validation data)

In [56]:
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, vertical_flip=True, rescale=1/255) 
## this is the data augmentor that we will use to try to improve model accuracy, with the range of parameters/allowed modifications it can perform on each image

train_data = datagen.flow_from_directory(
    directory = r'C:\Users\lucia\OneDrive - University of Calgary\University\Year 3\Semester 2\Bmen 415\Coursework\Course Project\GitHub\BMEN-415\Data\_split datasets\image train_test_val split\train',
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    seed = 112)
 
validation_data = datagen.flow_from_directory(
    r'C:\Users\lucia\OneDrive - University of Calgary\University\Year 3\Semester 2\Bmen 415\Coursework\Course Project\GitHub\BMEN-415\Data\_split datasets\image train_test_val split\val',
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    seed = 112)
 
model.fit(
    train_data,
    steps_per_epoch = 4487 // (batch_size*epochs),
    epochs = epochs,
    validation_data = validation_data,
    validation_steps = 642 // (batch_size*epochs))

# removed class weights since these seem to decrease model performance
# class_weight = {0: 4487/627, 1: 4487/44, 2: 4487/2240, 3: 4487/1568}) ## these class weights are based on the train dataset (total train samples/train samples for class i)

Found 4487 images belonging to 4 classes.
Found 642 images belonging to 4 classes.
Epoch 1/10
28/28 [==============================] - 19s 653ms/step - loss: 0.4910 - accuracy: 0.4375 - val_loss: 0.4243 - val_accuracy: 0.5625
Epoch 2/10
28/28 [==============================] - 17s 608ms/step - loss: 0.4977 - accuracy: 0.4152 - val_loss: 0.4391 - val_accuracy: 0.6094
Epoch 3/10
28/28 [==============================] - 14s 477ms/step - loss: 0.4568 - accuracy: 0.5201 - val_loss: 0.4485 - val_accuracy: 0.3594
Epoch 4/10
28/28 [==============================] - 15s 536ms/step - loss: 0.4736 - accuracy: 0.4732 - val_loss: 0.4264 - val_accuracy: 0.5781
Epoch 5/10
28/28 [==============================] - 17s 614ms/step - loss: 0.4510 - accuracy: 0.4844 - val_loss: 0.4260 - val_accuracy: 0.4688
Epoch 6/10
28/28 [==============================] - 15s 537ms/step - loss: 0.4522 - accuracy: 0.5112 - val_loss: 0.4251 - val_accuracy: 0.4844
Epoch 7/10
28/28 [==============================] - 17s 623

In [57]:
model.save('CNN_model.h5')

### Testing the model (on completely reserved test data)

In [59]:
from keras.models import load_model

test_data = tf.keras.utils.image_dataset_from_directory(
    r'C:\Users\lucia\OneDrive - University of Calgary\University\Year 3\Semester 2\Bmen 415\Coursework\Course Project\GitHub\BMEN-415\Data\_split datasets\image train_test_val split\test',
    image_size = (img_width, img_height),
    batch_size = batch_size,
    labels = 'inferred',
    label_mode = 'categorical',
    class_names = ['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented'],
    seed = 112)

# Load trained Keras model
model = load_model('CNN_model.h5')

# Evaluate model on test data
loss, accuracy = model.evaluate(test_data)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Found 1279 files belonging to 4 classes.
80/80 [==============================] - 9s 107ms/step - loss: 26.5733 - accuracy: 0.5317
Test loss: 26.573341369628906
Test accuracy: 0.5316653847694397
